In [109]:
import numpy as np
import lmfit
import wrapt
import inspect

scipy optimize

In [2]:
def residual(vars, x, data, eps_data):
    amp = vars[0]
    phaseshift = vars[1]
    freq = vars[2]
    decay = vars[3]

    model = amp * np.sin(x*freq + phaseshift) * np.exp(-x*x*decay)

    return (data-model) / eps_data

## lmfit

In [166]:
def scipy_to_lmfit_residual(var_order:list):
    """
    
    Parameters
    ----------
    var_order : list
        the variable order used by 
    
    TODO
    - make this a class so that it supports
      scipy_to_lmfit_residual(func, var_order)
      then it can be used to replace functions like
      func = scipy_to_lmfit_residual(func, var_order)
    - since using ObjectProxy, make it compatible with bound functions
      see https://wrapt.readthedocs.io/en/latest/wrappers.html#function-wrappers
    """

    class scipy_to_lmfit_residual(wrapt.ObjectProxy):

        def __init__(self, wrapped):
            super().__init__(wrapped)
            self.var_order = var_order
            return
        # /def

        def __call__(self, vars, *args, **kwargs):
            return self.__wrapped__(vars, *args, **kwargs)
        # /def

        def lmfit(self, params, *args, **kwargs):
            vars = [params[n].value for n in self.var_order]
            return self.__wrapped__(vars, *args, **kwargs)
        # /def
    
    return scipy_to_lmfit_residual

In [167]:
@scipy_to_lmfit_residual(['amp', 'phaseshift', 'freq', 'decay'])
def residual2(vars, x, data, eps_data):
    """test docstring"""
    amp = vars[0]
    phaseshift = vars[1]
    freq = vars[2]
    decay = vars[3]

    model = amp * np.sin(x*freq + phaseshift) * np.exp(-x*x*decay)

    return (data-model) / eps_data
# /def

In [168]:
lmfit_residual([0, 1, 2, 3], 10, 11, 1)

11.0

In [169]:
params = lmfit.Parameters()
params.add_many(
    ('amp', 0),
    ('phaseshift', 1),
    ('freq', 2),
    ('decay', 3),
)

In [170]:
residual2.lmfit(params, 10, 11, 1)

11.0